In [6]:
import datetime
import os
import time
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
from openpyxl import load_workbook

import Optim
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from datetime import datetime
import math

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, MultipleLocator

from hurst import compute_Hc

from layer import DenseGraphConv


np.seterr(divide='ignore',invalid='ignore')


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [56]:
# 각 값이 어떤 bin에 속하는지 인덱싱
def value_to_bin(value_list, m, M, increments):
    result = []
    for v in value_list:
        m_increments = m;
        bin_increments = 0
        while (m_increments < v):
            m_increments = m_increments + increments
            bin_increments = bin_increments + 1
        result.append(bin_increments)
    return result

# calculate omega
def error_cov_matrix(sigma, tau, P):
    matrix = np.diag(np.diag(P.dot(tau * sigma).dot(P.T)))
    return matrix


inc = 0.01
n_random = 25


def TE_caculate(X, increments=inc):
    
    # 최대값, 최소값, bin size
    M = np.max(X)
    M_ceil = np.ceil(M / increments) * increments  # 반올림
    m = np.min(X)
    m_floor = np.floor(m / increments) * increments  # 반내림

    # calculate TE
    TE_matrix = np.zeros((X.shape[1], X.shape[1]))
    for i in range(X.shape[1]):
        for j in range(X.shape[1]):
            if i == j:
                continue
            else:
                # print(v, '->', w)
                xn1 = [r[j] for r in X][1:]
                xn = [r[j] for r in X][:-1]
                yn = [r[i] for r in X][:-1]

                # binning
                xn1_bin = value_to_bin(xn1, m_floor, M_ceil, increments)
                xn_bin = value_to_bin(xn, m_floor, M_ceil, increments)
                yn_bin = value_to_bin(yn, m_floor, M_ceil, increments)

                # bin freq
                xn_freq = list(np.unique(np.array(xn_bin), return_counts=True))
                xn_freq.append(xn_freq[1] / sum(xn_freq[1]))
                xn_xn1_freq = list(np.unique(np.array([xn_bin, xn1_bin]).T, return_counts=True, axis=0))
                xn_xn1_freq.append(xn_xn1_freq[1] / sum(xn_xn1_freq[1]))
                xn_yn_freq = list(np.unique(np.array([xn_bin, yn_bin]).T, return_counts=True, axis=0))
                xn_yn_freq.append(xn_yn_freq[1] / sum(xn_yn_freq[1]))
                xn_xn1_yn_freq = list(np.unique(np.array([xn_bin, xn1_bin, yn_bin]).T, return_counts=True, axis=0))
                xn_xn1_yn_freq.append(xn_xn1_yn_freq[1] / sum(xn_xn1_yn_freq[1]))

                # calculate TE
                TE_xy = 0
                for ii, vv in enumerate(xn_xn1_yn_freq[0]):
                    p_in_in1_jn = xn_xn1_yn_freq[2][ii]
                    index_in_in1 = xn_xn1_freq[0].tolist().index(vv[:2].tolist())
                    p_in_in1 = xn_xn1_freq[2][index_in_in1]
                    index_in_jn = xn_yn_freq[0].tolist().index(vv[[0, 2]].tolist())
                    p_in_jn = xn_yn_freq[2][index_in_jn]
                    index_in = xn_freq[0].tolist().index(vv[0])
                    p_in = xn_freq[2][index_in]
                    TE_xy = TE_xy + p_in_in1_jn * np.log2(p_in_in1_jn * p_in / p_in_in1 / p_in_jn)
                TE_matrix[i, j] = TE_xy

    return TE_matrix


def ETE_caculate(X, TE_matrix, increments=inc):
    
    # 최대값, 최소값, bin size
    M = np.max(X);
    M_ceil = np.ceil(M / increments) * increments  # 반올림
    m = np.min(X);
    m_floor = np.floor(m / increments) * increments  # 반내림
    
    # calculate RTE
    RTE_matrix_all = np.zeros([X.shape[1], X.shape[1], n_random])
    for nn in range(n_random):
        # returns_shuffle = np.array(X).T.tolist()
        # random.shuffle(returns_shuffle)
        # returns_shuffle = np.array(returns_shuffle).T.tolist()
        RTE_matrix = np.zeros((X.shape[1], X.shape[1]))
        for i in range(X.shape[1]):
            for j in range(X.shape[1]):
                if i==j:
                    continue
                else:
                    xn1 = [r[j] for r in X][1:]
                    xn = [r[j] for r in X][:-1]
                    yn = [r[i] for r in X][:-1]
                    # binning
                    xn1_bin = value_to_bin(xn1, m_floor, M_ceil, increments)
                    xn_bin = value_to_bin(xn, m_floor, M_ceil, increments)
                    yn_bin = value_to_bin(yn, m_floor, M_ceil, increments)
                    np.random.shuffle(yn_bin)
    
                    # bin freq
                    xn_freq = list(np.unique(np.array(xn_bin), return_counts=True))
                    xn_freq.append(xn_freq[1]/sum(xn_freq[1]))
                    xn_xn1_freq = list(np.unique(np.array([xn_bin, xn1_bin]).T, return_counts=True, axis=0))
                    xn_xn1_freq.append(xn_xn1_freq[1]/sum(xn_xn1_freq[1]))
                    xn_yn_freq = list(np.unique(np.array([xn_bin, yn_bin]).T, return_counts=True, axis=0))
                    xn_yn_freq.append(xn_yn_freq[1]/sum(xn_yn_freq[1]))
                    xn_xn1_yn_freq = list(np.unique(np.array([xn_bin, xn1_bin, yn_bin]).T, return_counts=True, axis=0))
                    xn_xn1_yn_freq.append(xn_xn1_yn_freq[1]/sum(xn_xn1_yn_freq[1]))
                    # calculate RTE
                    RTE_xy = 0
                    for ii, vv in enumerate(xn_xn1_yn_freq[0]):
                        p_in_in1_jn = xn_xn1_yn_freq[2][ii]
                        index_in_in1 = xn_xn1_freq[0].tolist().index(vv[:2].tolist())
                        p_in_in1 = xn_xn1_freq[2][index_in_in1]
                        index_in_jn = xn_yn_freq[0].tolist().index(vv[[0, 2]].tolist())
                        p_in_jn = xn_yn_freq[2][index_in_jn]
                        index_in = xn_freq[0].tolist().index(vv[0])
                        p_in = xn_freq[2][index_in]
                        RTE_xy = RTE_xy + p_in_in1_jn * np.log2(p_in_in1_jn * p_in / p_in_in1 / p_in_jn)
                    RTE_matrix[i, j] = RTE_xy
        RTE_matrix_all[:,:,nn] = RTE_matrix
        # calculate ETE
    
    ETE_matrix = np.zeros((X.shape[1], X.shape[1]))
    for i in range(X.shape[1]) :
        for j in range(X.shape[1]):
            if i == j:
                continue
            TE = TE_matrix[i, j]
            rte_array = RTE_matrix_all[i, j, :]
            if TE - np.mean(rte_array) - np.std(rte_array) / n_random ** 0.5 > 0:
                ETE_matrix[i, j] = TE - np.mean(rte_array)
    
    return ETE_matrix

In [77]:
class TENet(nn.Module):
    def __init__(self, args, A, B, window_size):
        super(TENet, self).__init__()
        self.use_cuda = args.CUDA
        self.batch_size = args.BATCH_SIZE
        self.window_size = window_size

        # A와 B 초기화
        A = np.array(A, dtype=np.float32)
        A = A / np.sum(A)
        A_new = np.zeros((args.BATCH_SIZE, args.N_E, args.N_E), dtype=np.float32)
        for i in range(args.BATCH_SIZE):
            A_new[i, :, :] = A

        self.A = torch.from_numpy(A_new).cuda() if self.use_cuda else torch.from_numpy(A_new)
        self.adjs = [self.A]
        self.num_adjs = args.NUM_ADJ
        if self.num_adjs > 1:
            A = np.array(B, dtype=np.float32)
            A = A / np.sum(A, 1)
            A_new = np.zeros((args.BATCH_SIZE, args.N_E, args.N_E), dtype=np.float32)
            for i in range(args.BATCH_SIZE):
                A_new[i, :, :] = A

            self.B = torch.from_numpy(A_new).cuda() if self.use_cuda else torch.from_numpy(A_new)
            self.C = torch.from_numpy(A_new).cuda()
            self.adjs = [self.A,self.B,self.C]

        self.n_e = args.N_E

        # Conv2d 레이어
        self.conv1=nn.Conv2d(1, args.CHANNEL_SIZE, kernel_size = (1,args.K_SIZE[0]),stride=1)
        self.conv2=nn.Conv2d(1, args.CHANNEL_SIZE, kernel_size = (1,args.K_SIZE[1]),stride=1)
        self.conv3=nn.Conv2d(1, args.CHANNEL_SIZE, kernel_size = (1,args.K_SIZE[2]),stride=1)


        # Conv2d의 출력 차원 계산
        d= (len(args.K_SIZE)*(self.window_size) -sum(args.K_SIZE)+ len(args.K_SIZE))*args.CHANNEL_SIZE

        # GNN 레이어
        self.gnn1 = DenseGraphConv(d, args.HID1)
        self.gnn2 = DenseGraphConv(args.HID1, args.HID2)
        self.gnn3 = DenseGraphConv(args.HID2, 1)

        # Highway layer
        self.hw = args.HIGHWAY_WINDOW
        if self.hw > 0:
            self.highway = nn.Linear(self.hw, 1)

    def forward(self,x):
        c=x.permute(0,2,1)
        c=c.unsqueeze(1)
        # if self.decoder != 'GAT':
        a1=self.conv1(c).permute(0,2,1,3).reshape(self.batch_size,self.n_e,-1)
        a2=self.conv2(c).permute(0,2,1,3).reshape(self.batch_size,self.n_e,-1)
        a3=self.conv3(c).permute(0,2,1,3).reshape(self.batch_size,self.n_e,-1)

        x_conv = F.relu(torch.cat([a1, a2, a3], 2))
        # x_conv=F.relu(torch.cat([a1,a2,a3,a4,a5],2))
        x1 = F.relu(self.gnn1(x_conv,self.A))
        x2 = F.relu(self.gnn2(x1,self.A))
        x3 = self.gnn3(x2,self.A)
        x3 = x3.squeeze()
      

        if self.hw>0:
            z = x[:, -self.hw:, :]
            z = z.permute(0, 2, 1)
            z = self.highway(z)
            z = z.squeeze(2)
            x3 = x3 + z
        return x3



In [11]:

class WindowedDataset(Dataset):
    def __init__(self, data, data_y, window_size, horizon, scaler_x, scaler_y, volatility_type):
        self.data = data
        self.data_y = data_y
        self.window_size = window_size
        self.horizon = horizon
        self.scaler_x = scaler_x
        self.scaler_y = scaler_y
        self.volatility_type = volatility_type
    
    def __len__(self):
        return len(self.data) - self.window_size - self.horizon + 1
    
    def __getitem__(self, idx):
        # 데이터 슬라이스 추출
        window_data = self.data[idx:idx+self.window_size]
        label = self.data_y[idx]

        # 스케일링 (사전에 학습된 스케일러 사용)
        window_scaled = self.scaler_x.transform(window_data)
        label_scaled = self.scaler_y.transform(np.array([label]))
        
        return torch.tensor(window_scaled, dtype=torch.float32), torch.tensor(label_scaled.flatten(), dtype=torch.float32)


class ModelManager:
    def __init__(self, config, data, window_size, threshold, Effective, training_period):
        self.config = config
        self.window_size = window_size
        self.raw_data = data
        self.data_np = np.array(data.iloc[:, 1:])
        self.data_y = self.calculate_realized_volatility(self.data_np, window_size)  # 실현 변동성 계산
        self.training_period = training_period
        self.threshold = threshold
        self.Effective = Effective
        self.device = torch.device(config.DEVICE)
        self.model = None
        self.criterion = nn.MSELoss().to(self.device)
        self.optimizer = None
        self.A = None
        self.B = None
        self.results = []
        self.scaler_x = MinMaxScaler(feature_range=(0, 1))
        self.scaler_y = MinMaxScaler(feature_range=(0, 1))

        self.save_dir = None  # 저장할 디렉토리 경로

        self.test_total_loss_list = []
        self.hurst_values = []  # Hurst 지수를 저장하는 리스트
        self.phase_changes = []  # 국면 변화가 발생한 시점을 저장하는 리스트
        
        # 각 주식별로 손실 값을 저장하기 위한 구조
        self.test_each_loss_list = []

        
    def calculate_realized_volatility(self, data, window):
        realized_vols = []
        for i in range(window, len(data)):
            window_data = data[i-window:i]  # 30일치 데이터 슬라이스
            window_data = 100 * window_data
            squared_returns = np.square(window_data)  # 제곱
            realized_vol = np.sqrt(np.mean(squared_returns))  # 평균 계산 후 제곱근
            realized_vols.append(realized_vol)
        return np.array(realized_vols)

    def load_model(self, A, B, window_size):
        torch.cuda.empty_cache()
        return TENet(A, B, window_size).to(self.device)  # GNN 모델 생성

    def calculate_TE(self, data, Effective):
        if Effective:
            TE_matrix = TE_caculate(data)
            ETE_matrix = ETE_caculate(data, TE_matrix)
            return ETE_matrix
        else:
            TE_matrix = TE_caculate(data)
            return TE_matrix

    def update_A_B(self, matrix):
        self.A = matrix
        self.B = matrix

    def calculate_hurst(self, data):
        hurst_value = compute_Hc(data)[0]
        return hurst_value

    def check_phase_change(self, hurst_values, threshold=0.1):
        # 국면 전환 여부를 결정하는 함수
        if len(hurst_values) < 10:
            return False
        last_10 = hurst_values[-10:]
        mean_h = np.mean(last_10)
        std_h = np.std(last_10)
        return std_h > threshold  # 단순히 표준편차가 일정 임계값을 초과하면 국면 전환으로 간주

    def train_and_evaluate(self):
        self.window_size = window_size

        initial_data_x = self.data_np[:training_period]
        initial_data_y = self.data_y[:training_period-window_size]
        
        te_matrix = self.calculate_TE(initial_data_x, self.Effective)
        self.model = self.load_model(te_matrix, te_matrix, window_size)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.config.LR)
        
        self.train_model(initial_data_x, initial_data_y)
    

        for day in tqdm(range(training_period, len(self.data_np) - window_size + 1)):
            start_idx = max(0, day - training_period)
            current_data = self.data_np[start_idx:day]
            
            # Hurst 지수 계산
            hurst_value = self.calculate_hurst(current_data)
            self.hurst_values.append(hurst_value)
            
            # 과거 10일치 Hurst 지수를 이용한 국면 전환 체크
            if len(self.hurst_values) >= 10:
                last_10_days_h = self.hurst_values[-10:]
                if self.check_phase_change(last_10_days_h):
                    self.phase_changes.append(self.raw_data.index[day])

                    # 새로운 국면에서 TE 재계산 및 GNN 재학습
                    recent_3_year_data = self.data_np[day - training_period:day]
                    te_matrix = self.calculate_TE(recent_3_year_data, self.Effective)
                    self.model = self.load_model(te_matrix, te_matrix, window_size)
                    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.config.LR)
                    self.train_model(recent_3_year_data)
                
            else:
                print(f"Insufficient data to check phase change on day {day}.")

            
            TV_data_x = self.data_np[start_idx:day]
            TV_data_y = self.data_y[start_idx-window_size:day-window_size]
            
            # 매일 실현 변동성 예측 및 손실 계산
            test_data_x = self.data_x[day:day + window_size]
            test_data_y = self.data_y[day:day + window_size]




            # 매일 GNN 모델로 예측 및 손실 계산
            self.train(train_loader, valid_loader)
            test_total_loss, test_each_loss = self.evaluate_epoch(test_loader)
            self.test_total_loss_list.append(test_total_loss)
            self.test_each_loss_list.append(test_each_loss)

            # 국면 전환 여부 확인
            phase_changed = self.check_phase_change(self.hurst_values)
            if phase_changed:
                self.phase_changes.append(day)
                print(f"Phase change detected at index {day}. Reinitializing model.")

                # TE 재계산 및 모델 재학습
                te_matrix = self.calculate_TE(train_data_x, self.Effective)
                self.update_A_B(te_matrix)
                self.model = self.load_model(self.A, self.B, window_size)
                self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.config.LR)

            else:
                print(f"No phase change detected on day {day}, continuing with existing model.")



    def train_model(self, x, y):
        self.model.train()  # Set the model to training mode
        
        train_split = int(training_period * 0.7)
        train_x = x[:train_split]
        val_x = x[train_split:]
        train_y = y[:train_split]
        val_y = y[train_split:]
        
        self.scaler_x.fit(train_x)
        self.scaler_y.fit(train_y)
        
        # 학습 데이터, 검증 데이터, 테스트 데이터 준비
        train_loader = DataLoader(WindowedDataset(train_x, train_y, window_size, CONFIG.HORIZON, self.scaler_x, self.scaler_y), batch_size=self.config.BATCH_SIZE, shuffle=True,  drop_last=True)
        val_loader = DataLoader(WindowedDataset(val_x, val_y, window_size, CONFIG.HORIZON, self.scaler_x, self.scaler_y), batch_size=self.config.BATCH_SIZE, shuffle=False,  drop_last=True)
        
        best_val_loss = float('inf')
        best_model_state = None
    
        for epoch in range(CONFIG.EPOCHS):
            train_total_loss = self.train_epoch(train_loader)
            val_total_loss = self.evaluate_epoch(val_loader)            
            # validation loss가 최저일 때 모델을 저장
            if val_total_loss < best_val_loss:
                best_val_loss = val_total_loss
                best_model_state = self.model.state_dict()  # 모델의 가중치 저장
        
        self.model.load_state_dict(best_model_state)


    def train_epoch(self, train_loader):
        self.model.train()
        total_loss = 0
        each_loss = torch.zeros(self.data_x.shape[1]).to(self.device)  # 각 ETF에 대한 손실 초기화
        n_samples = 0
    
        for x, y in train_loader:
            x, y = x.to(self.device), y.to(self.device)
            self.optimizer.zero_grad()
            predictions = self.model(x)
            loss = self.criterion(predictions, y)
            loss.backward()
            self.optimizer.step()
    
            batch_size = x.size(0)
            total_loss += loss.item() * batch_size  # 배치의 총 손실을 샘플 수로 가중치 더함
            n_samples += batch_size
    
            # 각 ETF별 손실 합산
            for i in range(self.data_x.shape[1]):
                each_loss[i] += self.criterion(predictions[:, i:i + 1], y[:, i:i + 1]).item() * batch_size
    
        mse = each_loss / n_samples  # ETF별 평균 손실 계산
        total_loss = mse.mean().item()  # 전체 평균 손실 계산
    
        return total_loss, mse.cpu().numpy()

    def evaluate_epoch(self, loader):
        self.model.eval()
        total_loss = 0
        each_loss = torch.zeros(self.data_x.shape[1]).to(self.device)
        n_samples = 0  # 전체 샘플 수
    
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(self.device), y.to(self.device)
                predictions = self.model(x)
                loss = self.criterion(predictions, y)
                
                batch_size = x.size(0)
                total_loss += loss.item() * batch_size  # 배치의 총 손실을 샘플 수로 가중치 더함
                n_samples += batch_size
    
                # 각 ETF별 손실 합산
                for i in range(self.data_x.shape[1]):
                    each_loss[i] += self.criterion(predictions[:, i:i + 1], y[:, i:i + 1]).item() * batch_size
    
        mse = each_loss / n_samples  # ETF별 평균 손실 계산
        total_loss = mse.mean().item()  # 전체 평균 손실 계산
    
        return total_loss, mse.cpu().numpy()
    def create_save_directory(self):
        # 현재 날짜와 시간을 기반으로 폴더 이름 생성
        now = datetime.now().strftime('%Y%m%d_%H%M%S')
        folder_name = f"{self.Effective}_{self.volatility_type}_{self.window_size}_{self.training_period}_{self.testing_period}_{self.threshold}_{self.num_change_te}_{self.direction}_{now}"
        save_dir = os.path.join(os.getcwd(), folder_name)

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        self.save_dir = save_dir
        print(f"Results will be saved in: {save_dir}")

    def plot_results(self):
        if self.save_dir is None:
            self.create_save_directory()
    
        # 전체 손실을 플롯
        periods = range(1, len(self.train_total_loss_list) + 1)  # 1부터 시작하는 정수 시퀀스 생성
        plt.figure(figsize=(14, 7))  # 그래프 크기를 키움
        plt.plot(periods, self.train_total_loss_list, label='Train Loss')
        plt.plot(periods, self.val_total_loss_list, label='Valid Loss')
        plt.plot(periods, self.test_total_loss_list, label='Test Loss')
        for idx in self.te_update_indices:
            plt.axvline(x=idx, color='red', linestyle='--', label='TENet Update' if idx == self.te_update_indices[0] else "")
        plt.title('Total Loss Over Time')
        plt.xlabel('Period')
        plt.ylabel('Loss')
        plt.legend()
        plt.xticks(ticks=range(1, len(self.train_total_loss_list) + 1, 5), rotation=45)
        plt.grid(True)  # 눈금을 쉽게 볼 수 있도록 격자 표시
        plt.savefig(os.path.join(self.save_dir, 'total_loss.png'))  # 그래프를 파일로 저장
        # plt.show()
    
        # 주식별 손실을 플롯
        num_stocks = len(self.train_each_loss_list[0])  # 첫 번째 에포크의 주식 수를 기준으로 주식의 수 결정
    
        for stock_idx in range(num_stocks):
            plt.figure(figsize=(14, 7))  # 그래프 크기를 키움
            train_losses = [epoch_loss[stock_idx] for epoch_loss in self.train_each_loss_list]
            val_losses = [epoch_loss[stock_idx] for epoch_loss in self.val_each_loss_list]
            test_losses = [epoch_loss[stock_idx] for epoch_loss in self.test_each_loss_list]
    
            plt.plot(periods, train_losses, label=f'ETF {stock_idx+1} Train Loss')
            plt.plot(periods, val_losses, label=f'ETF {stock_idx+1} Valid Loss')
            plt.plot(periods, test_losses, label=f'ETF {stock_idx+1} Test Loss')
    
            plt.title(f'ETF {stock_idx+1} Loss Over Time')
            plt.xlabel('Period')
            plt.ylabel('Loss')
            plt.legend()
            plt.xticks(ticks=range(1, len(self.train_total_loss_list) + 1, 5), rotation=45)
            plt.grid(True)  # 눈금을 쉽게 볼 수 있도록 격자 표시
            plt.savefig(os.path.join(self.save_dir, f'ETF_{stock_idx+1}_loss.png'))  # 그래프를 파일로 저장
            # plt.show()
    
        # TE 변화를 플롯
        if self.TE_matrices:
            # Outflow 그래프
            plt.figure(figsize=(14, 7))  # 그래프 크기를 키움
            for stock_idx in range(len(self.TE_matrices[0])):
                outflow_te = [matrix[stock_idx, :].mean() for matrix in self.TE_matrices]
                plt.plot(periods, outflow_te, label=f'ETF {stock_idx+1} Outflow TE')
                
            # TE 업데이트 지점에 빨간 선 추가
            for idx in self.te_update_indices:
                plt.axvline(x=idx, color='red', linestyle='--', label='TE or ETE Update' if idx == self.te_update_indices[0] else "")
            plt.title('TE Changes Over Time (Outflow)')
            plt.xlabel('Period')
            plt.ylabel('TE Average')
            plt.legend()
            plt.xticks(ticks=range(1, len(self.train_total_loss_list) + 1, 5), rotation=45)
            plt.grid(True)
            plt.savefig(os.path.join(self.save_dir, 'te_outflow.png'))  # 그래프를 파일로 저장
            # plt.show()
    
            # Inflow 그래프
            plt.figure(figsize=(14, 7))  # 그래프 크기를 키움
            for stock_idx in range(len(self.TE_matrices[0])):
                inflow_te = [matrix[:, stock_idx].mean() for matrix in self.TE_matrices]
                plt.plot(periods, inflow_te, label=f'ETF {stock_idx+1} Inflow TE')
                
            # TE 업데이트 지점에 빨간 선 추가
            for idx in self.te_update_indices:
                plt.axvline(x=idx, color='red', linestyle='--', label='TE or ETE Update' if idx == self.te_update_indices[0] else "")    
            plt.title('TE Changes Over Time (Inflow)')
            plt.xlabel('Period')
            plt.ylabel('TE Average')
            plt.legend()
            plt.xticks(ticks=range(1, len(self.train_total_loss_list) + 1, 5), rotation=45)
            plt.grid(True)
            plt.savefig(os.path.join(self.save_dir, 'te_inflow.png'))  # 그래프를 파일로 저장
            # plt.show()
    
            # 전체 평균 TE 변화 그래프
            plt.figure(figsize=(14, 7))  # 그래프 크기를 키움
            avg_te = [matrix.mean() for matrix in self.TE_matrices]
            plt.plot(periods, avg_te, label='Overall TE Average')
            for idx in self.te_update_indices:
                plt.axvline(x=idx, color='red', linestyle='--', label='TE or ETE Update' if idx == self.te_update_indices[0] else "")
            plt.title('Overall TE Changes Over Time')
            plt.xlabel('Period')
            plt.ylabel('TE Average')
            plt.legend()
            plt.xticks(ticks=range(1, len(self.train_total_loss_list) + 1, 5), rotation=45)
            plt.grid(True)
            plt.savefig(os.path.join(self.save_dir, 'te_overall.png'))  # 그래프를 파일로 저장
            # plt.show()


    def save_results_to_excel(self):
        if self.save_dir is None:
            self.create_save_directory()

        filename = os.path.join(self.save_dir, f'Results_{self.threshold}_{self.num_change_te}_{self.direction}.xlsx')
        with pd.ExcelWriter(filename) as writer:
            # Losses
            pd.DataFrame(self.train_total_loss_list, columns=['Train Loss']).to_excel(writer, sheet_name='Train Loss')
            pd.DataFrame(self.val_total_loss_list, columns=['Validation Loss']).to_excel(writer, sheet_name='Validation Loss')
            pd.DataFrame(self.test_total_loss_list, columns=['Test Loss']).to_excel(writer, sheet_name='Test Loss')

            # ETF별 손실 저장
            num_stocks = len(self.train_each_loss_list[0])  # ETF 개수 (첫 번째 기간에서의 ETF 수를 기준으로 결정)
            periods = range(1, len(self.train_each_loss_list) + 1)  # 기간은 1부터 시작
    
            for stock_idx in range(num_stocks):
                df = pd.DataFrame({
                    'Period': periods,
                    'Train Loss': [period_loss[stock_idx] for period_loss in self.train_each_loss_list],
                    'Validation Loss': [period_loss[stock_idx] for period_loss in self.val_each_loss_list],
                    'Test Loss': [period_loss[stock_idx] for period_loss in self.test_each_loss_list]
                })
                df.to_excel(writer, sheet_name=f'ETF {stock_idx+1} Loss', index=False)

            # TE matrices
            for i, matrix in enumerate(self.TE_matrices):
                df = pd.DataFrame(matrix)
                if self.Effective :
                    df.to_excel(writer, sheet_name=f'Period {i+1} ETE')
                else : 
                    df.to_excel(writer, sheet_name=f'Period {i+1} TE')
                    
            # TE Update Indices
            if self.te_update_indices:
                pd.DataFrame(self.te_update_indices, columns=['TE Update Indices']).to_excel(writer, sheet_name='TE Update Indices')

    

In [7]:
class Config:
    # Data parameters
    N_E = 10
    MODEL = 'TENet'
    K_SIZE = [3, 5, 7]
    DECODER = 'GNN'
    HORIZON = 1
    NUM_ADJ = 1
    HIGHWAY_WINDOW = 0
    
    EPOCHS = 100
    BATCH_SIZE = 32
    
    
    SKIP_MODE = "none"
    ATTENTION_MODE = "naive"
    CHANNEL_SIZE = 12
    HID1 = 20
    HID2 = 30
    HIDRNN = 100
    RNN_LAYERS = 1
    HIDCNN = 12
    CNN_KERNEL = 6
    N_HEAD = 8
    D_K = 64
    D_V = 64
    SEED = 54321
    
    GPU = 0
    
    LOG_INTERVAL = 2000
    SAVE = 'model/model.pt'
    CUDA = True
    OPTIM = 'adam'
    LR = 0.001
    L1LOSS = False
    SKIP = 24
    HIDSKIP = 10
    NORMALIZE = 1
    OUTPUT_FUN = 'Linear'
    
    DEVICE = 'cuda:0'
    
    GCN_TRUE = True
    BUILDA_TRUE = True
    GCN_DEPTH = 2
    NUM_NODES = 8
    DROPOUT = 0.2
    SUBGRAPH_SIZE = 4
    NODE_DIM = 40
    DILATION_EXPONENTIAL = 2
    CONV_CHANNELS = 12
    RESIDUAL_CHANNELS = 12
    SKIP_CHANNELS = 32
    END_CHANNELS = 64
    IN_DIM = 1
    SEQ_IN_LEN = 32
    SEQ_OUT_LEN = 1
    LAYERS = 5
    WEIGHT_DECAY = 0.00001
    CLIP = 10
    PROPALPHA = 0.05
    TANHALPHA = 3
    NUM_SPLIT = 1
    STEP_SIZE = 100
    VALIDATION_FREQ=10

CONFIG = Config()


CONFIG.CUDA = torch.cuda.is_available()
if CONFIG.CUDA:
    torch.cuda.set_device(CONFIG.GPU)

In [ ]:
logR_df = pd.read_csv('data/log_df_etf10.csv')

manager = ModelManager(CONFIG, logR_df, window_size=30, training_period=250, threshold=None, Effective=False)

# 저장할 디렉토리 설정
manager.save_dir = "/"
                    
# 학습 및 평가
manager.train_and_evaluate()  # Example sizes in days
                    
# 결과 플롯 및 저장
manager.plot_results()
manager.save_results_to_excel()
                    

In [112]:
import os
import numpy as np

# Configuration and initialization
fin = open("/Users/sangheon/Desktop/CauGNN-main/TENet-master/data/log_df_etf10.txt")
data = np.loadtxt(fin, delimiter=',')

# 실험을 위한 파라미터 값 설정
window_sizes = [30, 60, 90]
training_periods = [750, 1500]
# num_change_tes = [4, 8]
# directions = ['outflow', 'inflow']
effectives = [False, True]

# For문을 통해 모든 조합에 대해 실험 수행
for window_size in tqdm(window_sizes, leave=False):
    for training_period in tqdm(training_periods, leave=False):
                for effective in tqdm(effectives, leave=False):
                    # 각 조합에 대해 디렉토리 생성
                    experiment_name = f"Window_{window_size}_Train_{training_period}_Effective_{effective}"
                    save_dir = os.path.join("Realized_experiments_change_all_period", experiment_name)
                    os.makedirs(save_dir, exist_ok=True)
                    
                    # ModelManager 객체 생성
                    manager = ModelManager(CONFIG, data, threshold=None, Effective=effective, volatility_type=2, num_change_te=None, direction=None)
                    
                    # 저장할 디렉토리 설정
                    manager.save_dir = save_dir
                    
                    # 학습 및 평가
                    manager.train_and_evaluate(window_size=window_size, training_period=250, testing_period=60)  # Example sizes in days
                    
                    # 결과 플롯 및 저장
                    manager.plot_results()
                    manager.save_results_to_excel()


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]


  0%|          | 0/85 [00:00<?, ?it/s]


  1%|          | 1/85 [00:04<05:44,  4.10s/it]


  2%|▏         | 2/85 [00:07<05:26,  3.94s/it]


  4%|▎         | 3/85 [00:11<05:21,  3.92s/it]


  5%|▍         | 4/85 [00:15<05:21,  3.97s/it]


                                     


KeyboardInterrupt: 

In [ ]:
# Configuration and initialization
fin = open("/Users/sangheon/Desktop/CauGNN-main/TENet-master/data/log_df_etf10.txt")
data = np.loadtxt(fin, delimiter=',')
manager = ModelManager(CONFIG, data, threshold=None, Effective=True, volatility_type=1, num_change_te=None, direction=None)
manager.train_and_evaluate(window_size=32, training_period=750, testing_period=250)  # Example sizes in days
manager.plot_results()
manager.save_results_to_excel()

In [ ]:
'''
변수
training period size : big window size of train
testing period size : big window size of test & big window moving size
window_size : small window moving size
threshold : TE or ETE threshold
volatility type : , 0 for log return, 1 for absolute log return, 2 for realized volatility

horizon : 1 (small window moving size) / Fixed
training rate : 0.7 in train set(on training period size) & Shuffled / Fixed
validation rate : 0.3 in train set / Fixed

changing point index(TE 임계값 넘는 지점)
changing point list
TE list
ETE list
Loss list of each stock
Loss list of all


특정 Threshold에 따라 모델 변경
전체 ETF의 TE 평균, inflow TE의 평균, outflow TE의 평균, inflow의 평균이 threshold를 넘는 ETF의 개수, outflow의 평균이 threshold를 넘는 ETF의 개수
총 5개의 경우

공통
window size : 3m, 6m, 1y, 3y, 5y
'''